In [ ]:
import igraph as ig
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import collections

## Facebook data

In [ ]:
df = pd.read_csv('facebook_combined.txt',sep=' ',names = ['a','b'])

tuples = [tuple(x) for x in df.values]
G = ig.Graph.TupleList(tuples)

## Karate data

In [ ]:
G = ig.read('karate.gml')

## Deezer europe network

In [ ]:
test = pd.read_csv('deezer_europe_edges.csv',sep=',')

tuples = [tuple(x) for x in test.values]
G = ig.Graph.TupleList(tuples)

## Pre-processing

In [ ]:
K = G.shell_index()
G.vs['shell'] = K

In [ ]:
nodes = []
for x in G.vs.indices:
    nodes.append(x)
    
global_core = [x for x in nodes if K[x] == max(K)]
global_code_name = [x['name'] for x in G.vs if x['shell'] == max(K)]

In [ ]:
clus = G.community_multilevel()
G.modularity(clus.membership)

In [ ]:
comm_count = 0
for x in [i for i in G.community_multilevel()]:
    comm_count += 1
    for node in x:
        G.vs[node]['community'] = comm_count
# comm_count

In [ ]:
count =0
fData = dict()
for num in range(1,comm_count+1): 
    data = dict()
    count += 1
    for i in [i for i in G.vs if i['community'] == num]:
        data[i['shell']] = data.get(i['shell'], 0) + 1
        fData[count] = data

In [ ]:
def avg_shell(x):
    sum_s=0
    sum_w=0
    for j in range(0,len(x)):
        sum_s=sum_s+x[j][0]*x[j][1]
        sum_w=sum_w+x[j][1]
    return(sum_s/sum_w)

In [ ]:
def std_div(x):
    deviations=0
    sum_e=0
    for i in range(0,len(x)):
        deviations = deviations + (((x[i][0] - avg_shell(x)) ** 2)*x[i][1] )
        sum_e=sum_e+x[i][1]
    variance = deviations / sum_e
    return((variance)**(1/2))

In [ ]:
def dispersion(x):
    return(((std_div(x))**2)/avg_shell(x))

In [ ]:
def com_size(x):
    a=0
    for i in range(0,len(x)):
        a=a+x[i][1]
    return(a)

## Entropy

In [ ]:
def getEntropy(i):
    test = [j for i,j in fData[i].items()]
    l = sum(test)
    tmp = []
    for i in test:
        dum = i/l
        tmp.append( dum * np.log2(dum))
    return -sum(tmp)
Entropy = [getEntropy(i) for i in range(1,comm_count+1)]

## Influence measure

In [ ]:
def getInfluence(commno):
    tmp = []
    for t in list(fData[commno].keys()):
        tmp.append(len([i for i in G.vs.indices if K[i] == t]))

    val = list(fData[commno].values())
    shel = list(fData[commno].keys())

    sumt= []
    for i in range(len(tmp)):
        sumt.append(shel[i]*val[i]/tmp[i])
    return sum(sumt)
Influence = [getInfluence(i) for i in range(1,comm_count+1)]

## Writing data into dataframe

In [ ]:
data=[]
for i in range(1,len(fData)+1):
    od = collections.OrderedDict(sorted(fData[i].items()))
    od = list(od.items())
    data.append([i,com_size(od),avg_shell(od),std_div(od),dispersion(od)])
d = pd.DataFrame(data,columns=["comm_no","comm_size","mean","std_dev","dispersion"])

In [ ]:
# Adding entropy to the dataframe
d['Entropy'] = Entropy
d['Influence'] = Influence

In [ ]:
d.sort_values('comm_size')

In [ ]:
# d.to_excel('Results_dr.xlsx',sheet_name='Sheet1')

## Shells vs Community nos along with their mean

In [ ]:
X = [len(j) for i,j in fData.items()]
Y = list(fData.keys())

X_1 = list(d['mean'])
X_1 = [i**4 for i in X_1]

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(X,Y,s=X_1)
# plt.scatter(X_1,Y)
plt.yticks(Y)
plt.xlabel('No of Shells covered')
plt.ylabel('Community Number')
# plt.legend(['No of Shells','Mean Shell Number'])
plt.savefig('ShellsCovered_vs_CommunityNo.jpg')

## No of nodes in each shell

In [ ]:
comm_num1 = 13
comm_num2 = 4
od_1 = collections.OrderedDict(sorted(fData[comm_num1].items()))
od_2 = collections.OrderedDict(sorted(fData[comm_num2].items()))

X1 = list(od_1.keys())
X2 = list(od_2.keys())
Y1 = list(od_1.values())
Y2 = list(od_2.values())

plt.figure(figsize=(10,5))
plt.xlabel('Shell Number')
plt.ylabel('Number of Nodes')
plt.plot(X1,Y1,label='Community_13')
plt.plot(X2,Y2,label='Community_4')
plt.legend()
plt.savefig('NodesinEachShell_Comapre.jpg')

## Avg shell no vs Dispersion Index

In [ ]:
X = list(d['dispersion'])
Y = list(d['mean'])
A = list(d['comm_size'])
A = [i for i in A]
colors = np.random.rand(16)

plt.figure(figsize=(10,5))
plt.xlabel('Dispersion Index')
plt.ylabel('Average Shell Number')
plt.scatter(X,Y,s=A, c=colors, alpha=0.5)
plt.savefig('DispersionIndex_vs_AverageShellNumber.jpg')

## Community Size vs Community No

In [ ]:
X = list(d['comm_size'])
Y = list(d['comm_no'])
A = list(d['mean'])
A = [i**4 for i in A]
plt.figure(figsize=(10,5))
plt.scatter(X,Y,s=A)
plt.xlabel('Community Size')
plt.ylabel('Community Number')
plt.savefig('CommunitynoVSCommunitySize.jpg')

## Shell_num vs no of nodes (for a particular community)

In [ ]:
# comm_num = 8

od = collections.OrderedDict(sorted(fData[comm_num].items()))
X = list(od.keys())
Y = list(od.values())

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(X,Y)
plt.xlabel('Shell Number')
plt.ylabel('Number of Nodes')
plt.savefig('ShellNumVSNumOfNodes.jpg')

## Shell num vs % of nodes present in the shell

In [ ]:
def getPlot(fData,i):
    od = collections.OrderedDict(sorted(fData[i].items()))
    od = list(od.items())
    
    shell_len = dict()
    for k in list(set(K)):
        shell_len[k] = len([i for i in G.vs if i['shell'] == k])
    
    r=[]
    for i in range(0,len(od)):
        r.append([od[i][0],(od[i][1])/shell_len[od[i][0]]])
        
    tmp = [i for i,j in r if j == 1]
    R = [(i,j) for i,j in r if i not in tmp]
    
    x = [i for i,j in R]
    y = [j for i,j in R]
    plt.figure(figsize=(15,5))
    plt.xticks(x,rotation=90)
    plt.xlabel('Shell Number')
    plt.ylabel('Fraction of nodes present')
#     plt.show()
    p = plt.plot(x,y)
    plt.savefig('test.{}.jpg'.format(i))
    return p

In [ ]:
getPlot(fData,4)
# getPlot(fData,8)

## Simple Graph

In [ ]:
def getSPlot(fData,i):
    od = collections.OrderedDict(sorted(fData[i].items()))
    od = list(od.items())
    
    shell_len = dict()
    for k in list(set(K)):
        shell_len[k] = len([i for i in G.vs if i['shell'] == k])
    
    r=[]
    for i in range(0,len(od)):
        r.append([od[i][0],(od[i][1])/shell_len[od[i][0]]])

    x = [i for i,j in r]
    y = [j for i,j in r]
    plt.figure(figsize=(15,7))
    plt.xlabel('Shell Number')
    plt.ylabel('Fraction of nodes present')
    plt.xticks(x)
#     plt.show()
    p = plt.plot(x,y)
    
    plt.savefig('testplot3.jpg')
    return p

In [ ]:
getSPlot(fData,3)

## Community Number vs Entropy

In [ ]:
Y = list(d['comm_no'])
X = list(d['Entropy'])
A = list(d['Influence'])
A = [i*10 for i in A]
colors = np.random.rand(comm_count)
plt.figure(figsize=(15,7))
plt.scatter(X,Y,s=A,c=colors,alpha=0.5)
plt.xlabel('Entropy')
plt.ylabel('Community Number')
plt.savefig('EntropyVSCommno_Dezzer.jpg')

## Entropy vs Mean shell number

In [ ]:
X = list(d['Entropy'])
Y = list(d['mean'])
plt.figure(figsize=(12,7))
plt.scatter(X,Y)
plt.xlabel('Entropy')
plt.ylabel('Weighted Mean')
plt.savefig('EntropyVSMeanShell_deezer.jpg')

## Jaccard coefficient

In [ ]:
def common_elements(a,b):
    a.sort()
    b.sort()
    i,j=0,0
    intersection=[ ]
    while i<len(a) and j<len(b):
        if a[i] == b[j]:
            intersection.append(a[i])
            i=i+1
            j=j+1
        elif a[i]<b[j]:
            i=i+1
        else:
            j=j+1
    return(len(intersection))

In [ ]:
jaccard = []
for i in range(0,comm_count):
    G_comm = G.subgraph(G.community_multilevel()[i])
    K_local = G_comm.shell_index()
    nodes_local = []
    for x in G_comm.vs:
        nodes_local.append(x)
    local_core = [x['name'] for x in nodes_local if K_local[x.index] == max(K_local)]
    U = len(global_code_name)
    V = len(local_core)
    intersection = common_elements(global_code_name,local_core)
    union = U + V - intersection
    jaccard.append(intersection/union)

## Global core and Local core properties

In [ ]:
fraction_of_community_nodes_in_local_core=[]
fraction_of_community_nodes_in_gobal_core=[]
fraction_of_global_core_in_local_core=[]
fraction_of_local_core_in_global_core=[]

for i in range(0,comm_count):
    G_comm = G.subgraph(G.community_multilevel()[i])
    K_local = G_comm.shell_index()
    G_comm.vs['shell'] = K_local
    nodes_local = []
    for y in G_comm.vs.indices:
        nodes_local.append(y)
    local_core = [G.community_multilevel()[i][x] for x in nodes_local if K_local[x] == max(K_local)]
    fraction_of_community_nodes_in_local_core.append(round(len(local_core)/len(G.community_multilevel()[i]),3))
    fraction_of_community_nodes_in_gobal_core.append(round(len(list(set(G.community_multilevel()[i]).intersection(global_core)))/len(G.community_multilevel()[i]),3))
    fraction_of_global_core_in_local_core.append(round(len(list(set(local_core).intersection(global_core)))/len(global_core),3))
    fraction_of_local_core_in_global_core.append(round(len(list(set(local_core).intersection(global_core)))/len(local_core),3))

In [ ]:
data=[]
for i in range(0,comm_count):    
    data.append([jaccard[i],fraction_of_community_nodes_in_local_core[i],fraction_of_community_nodes_in_gobal_core[i],fraction_of_global_core_in_local_core[i],fraction_of_local_core_in_global_core[i]])     
d = pd.DataFrame(data,columns=["jaccard","fract_comm_local","fract_comm_gobal","fract_global_local","fract_local_global"])
d.to_excel('task2_deezer.xlsx',sheet_name='Sheet1')

In [ ]:
d